# Train on differnet beta values and check if there is a phase transition.
# Here we use different number of training examples - similar effect should be seen.

In [1]:
import sys
import os
sys.path.append('Jerome_scripts_Original/PGM/')
sys.path.append('selfPackages/')
sys.path.append('Jerome_scripts_Original/')

import numpy as np
import scipy as sc
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

try:
    import rbm
    import bm
except:
    print 'Compiling cy_utilities first'
    curr_dir = os.getcwd()
    os.chdir('PGM/')
    !python setup.py build_ext --inplace
    print 'Compilation done'
    os.chdir(curr_dir)
    
import pickle
from ising import IsingModel
import estimate_correlations_ising
from sklearn.utils import shuffle
import inlineMovies

In [ ]:
import dill
dill.dump_session('./data_bruckner/data/notebookData/13_beta_vs_PeakWidth_PT-2.db')

In [ ]:
import dill
dill.load_session('./data_bruckner/data/notebookData/13_beta_vs_PeakWidth_PT-2.db')

In [2]:
def beta_from_corrLength(corrLen):
    beta = np.tan((np.exp(-1./corrLen)))
    return beta

In [13]:
#corrLengths = np.array([0.1,0.5,1.,5,10.,20.,50,100])
#corrLengths = np.linspace(0.01,20,10)
#beta = np.linspace(0.00001,1.2,10)
beta_r = np.array([10**-5,10**-4,10**-3,10**-2,10**-1,0.25,0.5,0.75,10**0,1.1])
print beta_r

[  1.00000000e-05   1.00000000e-04   1.00000000e-03   1.00000000e-02
   1.00000000e-01   2.50000000e-01   5.00000000e-01   7.50000000e-01
   1.00000000e+00   1.10000000e+00]


In [17]:
#Defining Parameters
Num_configs = np.ceil(10000*0.1/beta_r)

Num_configs = Num_configs[3:]

N = np.zeros(len(Num_configs)) + 100
Nstep = np.zeros(len(Num_configs)) + 200
print Num_configs

beta = np.zeros(len(Num_configs)) + 0.1

#Change the array type to int array from float array
N = N.astype(int)
Nstep = Nstep.astype(int)
Num_configs = Num_configs.astype(int)

[ 100000.   10000.    4000.    2000.    1334.    1000.     910.]


In [ ]:
%%capture captX

Xdata_big    = []
Xresults_big = []

XBM      = []
Xdata    = []
Xresults = []
XRBM     = []

Num_reps = 1

for j in range(Num_reps):
    
    Xdata_big.append(Xdata)
    Xresults_big.append(Xresults)
    
    XBM      = []
    Xdata    = []
    Xresults = []
    XRBM     = []
    
    for i in np.arange(len(beta)):
        #Generate spins using Gibbs Sampling MCMC
        XBM.append(bm.BM(N=N[i],nature='Spin'))

        #Define Couplings
        XBM[i].layer.couplings =  beta[i]*(np.eye(N=N[i],k=1) + np.eye(N=N[i],k=-1) )
        XBM[i].layer.couplings[0,-1] += beta[i]
        XBM[i].layer.couplings[-1,0] += beta[i]

        #Generating the ising model data
        Xdata.append(XBM[i].gen_data(Nchains=1,Nthermalize=0,Lchains=Num_configs[i],Nstep=Nstep[i])) # N steps.

        #Train the model
        XRBM.append(rbm.RBM(visible = 'Spin', n_v=N[i], n_h = 1, hidden = 'Spin',zero_field = True))
        Xresults.append(XRBM[i].fit(Xdata[i],n_iter=250,verbose=0,learning_rate = 0.001,record=['W'],record_interval=100,batch_size=10,N_MC= 20, lr_decay=False,batch_norm=False))

In [ ]:
# Top Eignvectors of the C_ij matrix:
def topEV(data):
    #Find the covariance matrix:
    covMAT = np.cov(data)
    Eval,Evec = np.linalg.eig(covMAT)
    return Evec(:,argmax(Eval))